# Plot utils

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Rectangle
from IPython.display import HTML

def draw_bin(x):
    xmin = x - .5
    xmax = x + .5
    plt.plot([xmin, xmax], [0, 0], c='k', lw=3)
    plt.plot([xmin, xmin], [0, 1], c='k', lw=3)
    plt.plot([xmax, xmax], [0, 1], c='k', lw=3)

def draw_worker(x, y):
    plt.plot([x, x], [y, 2], c='b', lw=5)
    plt.scatter(x, y + .02, marker='^', c='b', s=200, zorder=2)

box_width = .5
box_height = .25
def draw_box(x, y):
    box = Rectangle([x - box_width / 2, y], box_width, box_height, color='sandybrown', ec='k')
    plt.gca().add_patch(box)

def draw_scene(worker, box):
    draw_bin(0)
    draw_bin(2)
    draw_worker(*worker)
    draw_box(*box)

def animate(workers, boxes):
    fig, ax = plt.subplots()
    def draw_frame(i):
        plt.clf()
        draw_scene(workers[i], boxes[i])
    ani = FuncAnimation(fig, draw_frame, frames=len(workers), interval=1e3)
    display(HTML(ani.to_jshtml()))
    plt.close()

In [ ]:
workers = [(1, 1.5), (0, 1.5), (0, .25), (0,  1.5), (2, 1.5), (2, .25), (2, 1.5), (1, 1.5)]
boxes   = [(0,   0), (0,   0), (0,   0), (0, 1.25), (2, 1.25), (2,  0), (2,   0), (2,   0)]
animate(workers, boxes)

# Skills

- **top**
- **left**
- **right**
- **grasp**
- **release**

In [ ]:
def add_states(v, n):
    states = []
    for i in range(n):
        w = v.add_variable(2)
        b = v.add_variable(2)
        states.extend([w, b])
    for state in states:
        v.add_constraint(state >= [-.5, 0])
        v.add_constraint(state <= [2.5, 2])
    return states

def top(gcs, label, grasp, direction):
    v = gcs.add_vertex('top_' + label)
    w0, b0, w1, b1, w2, b2 = add_states(v, 3)
    v.add_cost(cp.norm1(w1 - w0))
    v.add_cost(cp.norm1(w2 - w1))
    if grasp:
        v.add_constraint(w1 - b1 == w0 - b0)
        v.add_constraint(w2 - b2 == w0 - b0)
        v.add_constraint(w1[1] >= 1 + box_height)
        v.add_constraint(w2[1] >= 1 + box_height)
    else:
        v.add_constraint(b1 == b0)
        v.add_constraint(b2 == b0)
        v.add_constraint(w1[1] >= 1)
        v.add_constraint(w2[1] >= 1)
    if direction == 'up':
        v.add_constraint(w0[0] == w1[0])
        v.add_constraint(w1[1] == w2[1])
    elif direction =='down':
        v.add_constraint(w0[1] == w1[1])
        v.add_constraint(w1[0] == w2[0])
    else:
        raise ValueError
    return v
    
def left(gcs, label, grasp):
    v = gcs.add_vertex('left_' + label)
    w0, b0, w1, b1 = add_states(v, 2)
    v.add_cost(cp.norm1(w1 - w0))
    v.add_constraint(w0 <= [.5, 1 + box_height])
    v.add_constraint(w1 <= [.5, 1 + box_height])
    if grasp:
        v.add_constraint(w1 - b1 == w0 - b0)
    else:
        v.add_constraint(b1 == b0)
    return v

def right(gcs, label, grasp):
    v = gcs.add_vertex('right_' + label)
    w0, b0, w1, b1 = add_states(v, 2)
    v.add_cost(cp.norm1(w1 - w0))
    v.add_constraint(w0[0] >= 1.5)
    v.add_constraint(w0[1] <= 1 + box_height)
    v.add_constraint(w1[0] >= 1.5)
    v.add_constraint(w1[1] <= 1 + box_height)
    if grasp:
        v.add_constraint(w1 - b1 == w0 - b0)
    else:
        v.add_constraint(b1 == b0)
    return v

def grasp_release(gcs, label):
    v = gcs.add_vertex(label)
    w0, b0, w1, b1 = add_states(v, 2)
    v.add_constraint(w0[0] >= b0[0] - box_width / 2)
    v.add_constraint(w0[0] <= b0[0] + box_width / 2)
    v.add_constraint(w0[1] == b0[1] + box_height)
    v.add_constraint(w1 == w0)
    v.add_constraint(b1 == b0)
    v.add_constraint(w0 <= [2.5, 1])
    v.add_constraint(b0 <= [2.5, 1])
    return v

def add_edge(gcs, v, w):
    e = gcs.add_edge(v, w)
    e.add_constraint(v.variables[-2] == w.variables[0])
    e.add_constraint(v.variables[-1] == w.variables[1])
    return e

# GCS

In [ ]:
initial_state = [(2, 1.5), (0, 0)]
final_state = [(0, 1.5), (2, 0)]

In [ ]:
import cvxpy as cp
import numpy as np
from gcspy import GraphOfConvexSets

In [ ]:
gcs = GraphOfConvexSets()

s = gcs.add_vertex('s')
ws = s.add_variable(2)
bs = s.add_variable(2)
s.add_constraint(ws == initial_state[0])
s.add_constraint(bs == initial_state[1])

t = gcs.add_vertex('t')
wt = t.add_variable(2)
bt = t.add_variable(2)
t.add_constraint(wt == final_state[0])
t.add_constraint(bt == final_state[1])

top0 = top(gcs, '0', 0, 'down')
left0 = left(gcs, '0', 0)
right0 = right(gcs, '0', 0)

add_edge(gcs, s, top0)
add_edge(gcs, s, left0)
add_edge(gcs, s, right0)
add_edge(gcs, top0, left0)
add_edge(gcs, top0, right0)

grasp = grasp_release(gcs, 'grasp')
add_edge(gcs, left0, grasp)
add_edge(gcs, right0, grasp)

top1 = top(gcs, '1', 1, 'up')
left1 = left(gcs, '1', 1)
right1 = right(gcs, '1', 1)

add_edge(gcs, grasp, top1)
add_edge(gcs, grasp, left1)
add_edge(gcs, grasp, right1)
add_edge(gcs, top1, left1)
add_edge(gcs, top1, right1)

release = grasp_release(gcs, 'release')
add_edge(gcs, left1, release)
add_edge(gcs, right1, release)

top2 = top(gcs, '2', 0, 'up')
left2 = left(gcs, '2', 0)
right2 = right(gcs, '2', 0)
add_edge(gcs, release, top2)
add_edge(gcs, release, left2)
add_edge(gcs, release, right2)

add_edge(gcs, top2, t)
add_edge(gcs, left2, t)
add_edge(gcs, right2, t)

In [ ]:
gcs.graphviz()

In [ ]:
prob = gcs.solve_shortest_path(s, t)
print('Problem status:', prob.status)
print('Optimal value:', prob.value)

In [ ]:
path = [s]
workers = [initial_state[0]]
boxes = [initial_state[1]]
while path[-1] != t:
    for e in gcs.outgoing_edges(path[-1]):
        ye = e.y.value
        if ye is not None and ye > .5:
            path.append(e.head)
            for i, v in enumerate(e.head.variables[2:]):
                if i%2:
                    boxes.append(v.value)
                else:
                    workers.append(v.value)
            break

In [ ]:
animate(workers, boxes)